# Homework 07

#### Part Two-Scrape the content of https://www.lemonde.fr/en/ to an external site. and save it as a CSV
#### We want: titles, subhead, article URL, whether it's premium or not, byline, article type, image URL.

In [60]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.lemonde.fr/en/")
doc = BeautifulSoup(response.text)

In [61]:
#doc.find('title').text

In [62]:
articles = doc.find_all("div", class_="article")
articles[0]

<div class="article article--main"> <a href="https://www.lemonde.fr/en/environment/article/2025/06/11/environmental-transition-the-two-faces-of-emmanuel-macron_6742252_114.html"> <span class="icon__premium icon--outside"><span class="sr-only">Subscribers only</span></span><h1 class="article__title"></h1><p class="article__title-label">Environmental transition: The two faces of Emmanuel Macron</p> <div class="article__media-container"> <picture class="article__media"> <img alt="Emmanuel Macron on the program " france="" height="2" in="" june="" loading="eager" nice="" oc="" on="" sizes="(min-width: 1024px) 421px, 100vw" src="https://img.lemde.fr/2025/06/11/0/0/8256/5504/400/266/75/0/64dda24_sirius-fs-upload-1-o2z7pgbpwop7-1749622108697-dsc-7257.jpg" srcset=" https://img.lemde.fr/2025/06/11/0/0/8256/5504/800/533/75/0/64dda24_sirius-fs-upload-1-o2z7pgbpwop7-1749622108697-dsc-7257.jpg 400w, https://img.lemde.fr/2025/06/11/0/0/8256/5504/1000/666/75/0/64dda24_sirius-fs-upload-1-o2z7pgbpwop7-

In [63]:
rows = []

articles = doc.find_all("div", class_="article")

for article in articles:
    try:
        # URL
        a_tag = article.find("a", href=True)
        if not a_tag:
            continue
        href = a_tag["href"]
        article_url = "https://www.lemonde.fr" + href if href.startswith("/") else href
        
        # titles
        title_tag = article.find(class_="article__title")
        title = title_tag.get_text(strip=True) if title_tag else "N/A"
        
        # subhead
        subhead_tag = article.find("p", class_="article__desc")
        subhead = subhead_tag.get_text(strip=True) if subhead_tag else "N/A"
        
        # whether it's premium or not
        is_premium = "Premium" if article.find(class_="icon__premium") else "not Premium"
        
        # byline
        byline_tag = article.find("p", class_="article__byline")
        byline = byline_tag.get_text(strip=True) if byline_tag else "N/A"
        
        # article type
        type_tag = article.find("p", class_="article__type")
        article_type = type_tag.get_text(strip=True) if type_tag else "N/A"
        
        # image URL
        img_tag = article.find("img")
        img_url = img_tag["src"] if img_tag and "src" in img_tag.attrs else "N/A"
        
        rows.append({
            "title": title,
            "subhead": subhead,
            "article URL": article_url,
            "premium": is_premium,
            "byline": byline,
            "article type": article_type,
            "image URL": img_url
        })
    except Exception as e:
        print(f"error: {e}")
        continue

In [64]:
import pandas as pd
df = pd.json_normalize(rows)
df.head()

,title,subhead,article URL,premium,byline,article type,image URL
0,,"Flying the flag for environmental action, as h...",https://www.lemonde.fr/en/environment/article/...,Premium,N/A,N/A,https://img.lemde.fr/2025/06/11/0/0/8256/5504/...
1,"Brian Wilson, one of pop music's great visiona...",N/A,https://www.lemonde.fr/en/culture/article/2025...,Premium,N/A,N/A,N/A
2,Harvey Weinstein found guilty of sexual assaul...,N/A,https://www.lemonde.fr/en/international/articl...,not Premium,N/A,N/A,N/A
3,Gavin Newsom delivers scathing criticism of Tr...,As the mayor of Los Angeles imposed a curfew a...,https://www.lemonde.fr/en/international/articl...,Premium,N/A,N/A,N/A
4,Colombia gripped by waves of violence,A series of attacks has plunged two southweste...,https://www.lemonde.fr/en/international/articl...,Premium,N/A,N/A,N/A


In [65]:
df.to_csv("lemonde_articles.csv", index=False)